In [1]:
from ngram_tools.download_ngrams import download_ngram_files
from ngram_tools.convert_to_jsonl import convert_to_jsonl_files
from ngram_tools.lowercase_ngrams import lowercase_ngrams
from ngram_tools.lemmatize_ngrams import lemmatize_ngrams
from ngram_tools.filter_ngrams import filter_ngrams
from ngram_tools.sort_ngrams import sort_ngrams
from ngram_tools.consolidate_ngrams import consolidate_duplicate_ngrams
from ngram_tools.index_and_create_vocab import index_and_create_vocab_files

from ngram_tools.helpers.verify_sort import check_file_sorted
from ngram_tools.helpers.print_jsonl_lines import print_jsonl_lines

ModuleNotFoundError: No module named 'ngram_tools'

# **Process Unigrams to Generate a Vocabulary File**

## **Goal**: Make a list of the _n_ most common unigrams for later use filtering multigrams

This workflow is resource-intensive and is probably only practical when run on a computing cluster. On my university's High Performance Computing (HPC) cluster, I request the maximum 14 cores (48 logical processors) and 128G of memory and use a 2T fast-I/O NVMe SSD filespace—and I still run up against time and resource limits. I've designed the code to be efficient, although further optimization is surely possible.

The code affords options to keep conserve resources. Throughout the workflow you can specify `compress=True`, which tells a script to compress its output files. In my experience, there is little downside to using LZ4 compression, since it's very fast and cuts file sizes by about half. Downstream modules will see the `.lz4` extensions and handle the files accordingly. If you know your workflow runs correctly and wish to further conserve space, you can specify `delete_input=True` for many of the scripts; this will delete the source files for a given step once it is complete. The scripts are fairly memory-efficient—with the exception of `sort_ngrams` and `index_and_create_vocab_files`, which sort multiple files in memory at once. When processing multigrams, I've found that allocating more than ~10 workers in these scripts leads to memory exhaustion (with 128G!) and slow processing.

`[Total runtime with compression:  0:43:54.8]`

### Download unigrams
Here, I'm using the `download_ngrams` module to fetch unigrams appended with part-of-speech (POS) tags (e.g., `_VERB`). Although you can specify `ngram_type='untagged'`, POS tags are necessary to lemmatize the tokens. Specify the number of parallel processes you wish to use by setting `workers` (the default is all available processors).

`[Runtime with compression:  0:00:39.639428]`

In [ ]:
download_ngram_files(
    ngram_size=1,
    ngram_type='tagged',
    file_range=(0, 24),
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False
)

### Convert files from TXT to JSONL
This module converts the original unigram files' text data to a more flexible JSON Lines (JSONL) format. Although this increases storage demands, it makes downstream processing more efficient.

`[Runtime with compression:  0:02:17.890276]`

In [ ]:
convert_to_jsonl_files(
    ngram_size=1,
    ngram_type='tagged',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False,
    delete_input=True
)

### Make unigrams all lowercase
This module lowercases all characters in the unigrams. Most use cases benefit from this.

`[Runtime with compression:  0:01:04.142135]`

In [ ]:
lowercase_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False,
    delete_input=True
)

### Lemmatize the unigrams
This module lemmatizes the unigrams—that is, reduces them to their base forms. This is desirable for most use cases. Example: `people_NOUN` ("the people of this land") will be converted to `person` in the output; `people_VERB` ("to people this land") will not. My code uses the [NLTK Lemmatizer](https://www.nltk.org/api/nltk.stem.WordNetLemmatizer.html?highlight=wordnet), which requires requires POS-tagged unigrams. The tags are discarded after lemmatization as they're no longer useful, saving space.

`[Runtime with compression:  0:01:22.440506]`

In [ ]:
lemmatize_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False,
    delete_input=True
)

### Filter the unigrams
This module removes tokens that provide little information about words' semantic context: those that contain numerals (`numerals=True`), nonalphabetic characters (`nonalpha=True`), stopwords (high-frequency, low information tokens like "the" and "into"; `stops=True`), or short words (those below a certain user-specified character count; here, `min_token_length=3`).

You can also specify `min_tokens`—the minumum length of a retained ngram after filtering its tokens. This is mainly intended for use when processing multigrams. However, it's still good to specify `min_tokens=1` for unigrams, as it completely discards the data for any unigram violating our criteria. Empty output files will be deleted.

`[Runtime with compression:  0:00:59.650950]`

In [ ]:
filter_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    numerals=True,
    nonalpha=True,
    stops=True,
    min_token_length=3,
    min_tokens=1,
    compress=True,
    overwrite=False,
    delete_input=True
)

### Sort and combine the unigram files
This modules creates a single, fully-sorted unigram file out of the filtered files. This is crucial for the next step (ngram consolidation; see below).   

Sorting a giant file is a resource-hungry process and I've tried to implement an efficient approach that leverages parallelism: We first sort the filtered files in parallel using Python's standard sorting algorithm [Timsort](https://en.wikipedia.org/wiki/Timsort); then, we incrementally [heapsort](https://en.wikipedia.org/wiki/Heapsort) the files in parallel until we get down to 2 files. Finally, we heapsort the final 2 files (necessarily using one processor) to arrive at a single combined and sorted unigram file.

`[Runtime with compression:  0:17:54.962300]`

In [ ]:
sort_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    workers=12,
    sort_key='ngram',
    compress=True,
    overwrite=False,
    sort_order='ascending',
    delete_input=True
)

### Verify sort [OPTIONAL]
If we want, we can verify that the output file is correctly sorted. If the script outputs `True`, then the file is sorted. Bear in mind that you need to specify the file path manually here; be sure to use the right file extension based on whether `sort_ngrams` was run with `compress=True`.

In [ ]:
check_file_sorted(
    input_file=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng/'
        '1gram_files/6corpus/1gram-merged.jsonl.lz4'
    ),
    field="ngram",
    sort_order="ascending"
)

### Consolidate duplicate unigrams
This module consolidates the sorted unigram file. Lowercasing and lemmatizing produce duplicate unigrams. Now that the file is sorted, we can scan through it and consolidate consecutive idential duplicates. This involves summing their overall and yearly frequencies and document counts. It also leads to a much smaller file.

`[Runtime with compression:  0:07:33.662163`

In [ ]:
consolidate_duplicate_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    lines_per_chunk=1000000,
    compress=True,
    overwrite=False
)

### Index unigrams and create vocabulary file
Most use cases will require an indexed list of "valid" (i.e., reasonably common) vocabulary words. This indexing script serves the dual functions of (1) mapping each unigram to an index number (saved in `/6corpus/1gram-corpus-indexed.jsonl`) and (2) culling this file into a vocabulary list consisting of the _n_ most frequent unigrams (saved in `6corpus/1gram-corpus-vocab_list_match.txt`). The vocabulary file provides a critical means of filtering excessively rare words out of the corpus. Unlike files upstream in the workflow, the vocabulary files are not large and don't need to be compressed.

`[Runtime with compression:  0:12:02.432830]`

In [ ]:
index_and_create_vocab_files(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False,
    vocab_n=80000
)

### Verify indexing [OPTIONAL]
We can verify that the final indexed file looks right.

In [ ]:
print_jsonl_lines(
    file_path=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng/'
        '1gram_files/6corpus/1gram-corpus-indexed.jsonl.lz4'
    ),
    start_line=79998,
    end_line=79999,
    parse_json=True
)